# Install PheWAS

In [ ]:
install.packages("devtools")
devtools::install_github("PheWAS/PheWAS")


# Load Libraries

In [ ]:
library(tidyverse)  # Data wrangling packages.
library(dplyr)
library(parallel)
library(PheWAS)

# Set your Variables

In [ ]:
rs_name <- 'rs199768005'
pop_variable <- 'all'
i <- sprintf('as Cov Sex+ 1-3 principals_comp+ age+ %s with all Diseases',pop_variable)
print(i)

# PheWAS analysis with covariates : Sex, Age and 1-3 PCA, cores 94

In [ ]:
name_of_file_in_bucket <- sprintf('data/%s_%s_Genotype_infomation.csv',pop_variable,rs_name)
# Get the bucket name
my_bucket <- Sys.getenv('WORKSPACE_BUCKET/data')
#######################__________________#####################___________________###########
# Copy the file from current workspace to the bucket
system(paste0("gsutil cp ", my_bucket, "/", name_of_file_in_bucket, " ."), intern=T)
# Load the file into a dataframe
genotypes <- read.csv(name_of_file_in_bucket,sep=',')
head(genotypes)
name_of_file_in_bucket <- sprintf('data/%s_%s_Conditions.csv',pop_variable,rs_name)
      # Get the bucket name
      my_bucket <- Sys.getenv('WORKSPACE_BUCKET')
      # Copy the file from current workspace to the bucket
      system(paste0("gsutil cp ", my_bucket, "/", name_of_file_in_bucket, " ."), intern=T)
      # Load the file into a dataframe
      icd10cm_codes=read_csv(name_of_file_in_bucket ,col_types="ifci")
head(icd10cm_codes)
phenotypes=createPhenotypes(icd10cm_codes,aggregate.fun=sum,min.code.count = 2)
head(phenotypes)
#######################__________________#####################___________________###########
data=inner_join(phenotypes,genotypes)
head(data)
#######################__________________#####################___________________###########
results=phewas_ext(data,phenotypes=names(phenotypes)[-1],genotypes=c(rs_name),covariates=c("sex",'age','principal_component_1','principal_component_2','principal_component_3'), cores=94)
results_d=addPhecodeInfo(results)
#List the significant results
sig_results <- results_d[results_d$bonferroni&!is.na(results_d$p),]
#######################__________________#####################___________________###########
# if you want to save data for more analysis in python
# # Replace df with THE NAME OF YOUR DATAFRAME
# my_dataframe <- data
# # Replace 'test.csv' with THE NAME of the file you're going to store in the bucket (don't delete the quotation marks)
# destination_filename <- sprintf('data/%s_PheCodes.csv',pop_variable)
# ########################################################################
# ##
# ################# DON'T CHANGE FROM HERE ###############################
# ##
# ########################################################################
# # store the dataframe in current workspace
# write_excel_csv(my_dataframe, destination_filename)
# # Get the bucket name
# my_bucket <- Sys.getenv('WORKSPACE_BUCKET')
# # Copy the file from current workspace to the bucket
# system(paste0("gsutil cp ./", destination_filename, " ", my_bucket, "/data/"), intern=T)
#######################__________________#####################___________________###########
print('Complete!!!')

# Create Plots

In [ ]:
phewas_plot <- phewasManhattan(results,OR.direction = T,title=sprintf("PheWAS Manhattan Plot %s ",rs_name), annotate.size=5.6,point.size = 4)
phewas_plot <- phewas_plot + theme(panel.grid.major = element_blank(), panel.grid.minor = element_blank())
# save plot
png(sprintf("./new_run/%s_%s_phewas_GQ_40_icd10_icd9_C_2_1_%s.png",rs_name,pop_variable,i), width=1200, height=800)
print(phewas_plot)
dev.off()
# show plot
phewas_plot
# save the files
head(results_d)
my_dataframe <- results
destination_filename <- sprintf('new_run/%s_%s_phewas_GQ_40_icd10_9_C_2_results_%s.csv',rs_name,pop_variable,i)
# store the dataframe in current workspace
write_excel_csv(my_dataframe, destination_filename)
# Get the bucket name
my_bucket <- Sys.getenv('WORKSPACE_BUCKET')
# Copy the file from current workspace to the bucket
system(paste0("gsutil cp ./", destination_filename, " ", my_bucket, "/data/"), intern=T)
my_dataframe <- results_d
destination_filename <- sprintf('new_run/%s_%s_phewas_GQ_40_icd10_9_C_2_results_d_%s.csv',rs_name,pop_variable,i)
# store the dataframe in current workspace
write_excel_csv(my_dataframe, destination_filename)
# Get the bucket name
system(paste0("gsutil cp ./", destination_filename, " ", my_bucket, "/data/"), intern=T)
system(paste0("gsutil cp ./", destination_filename, " ", my_bucket, "/data/"), intern=T)
phewas_plot <- phewasManhattan(results,OR.direction = T, title=sprintf("PheWAS Manhattan Plot %s ",rs_name), annotate.size=4,point.size = 3)
phewas_plot <- phewas_plot + theme(panel.grid.major = element_blank(), panel.grid.minor = element_blank())
# save plot
png(sprintf("./new_run/%s_%s_phewas_GQ_40_icd10_icd9_C_2_%s.png",rs_name,pop_variable,i), width=1200, height=800)
dev.off()
print(phewas_plot)
#######################__________________#####################___________________###########

#######################__________________#####################___________________###########
pdf(file = sprintf("./new_run/%s_Phewas_%s_1.pdf", rs_name, i), width = 18, height = 14)

# Step 2: Create the plot with R code

options(repr.plot.res = 300) # Adjust DPI as needed

# Create PheWAS Manhattan Plot
phewas_plot <- phewasManhattan(results, 
                               OR.direction = TRUE, 
                               title = sprintf("PheWAS Manhattan Plot %s", rs_name), 
                               annotate.size = 6, 
                               point.size = 5)

# Adjust x-axis label size and rotation
phewas_plot <- phewas_plot + theme(axis.text.x = element_text(size = 15, angle = 90))
# phewas_plot <- phewas_plot + geom_hline(yintercept = 4.4, linetype = "solid", color = "red", alpha = 0.5, size = 2)
# Remove gridlines
phewas_plot <- phewas_plot + theme(panel.grid.major = element_blank(), 
                                   panel.grid.minor = element_blank())

# Adjust y-axis limits
phewas_plot <- phewas_plot + coord_cartesian(ylim = c(0, 5))

# Step 3: Run dev.off() to create the file!
print(phewas_plot)
dev.off()
#######################__________________#####################___________________###########




In [ ]:
pdf(file = sprintf("./new_run/%s_Phewas_%s_2.pdf", rs_name, i), width = 18, height = 14)

# Step 2: Create the plot with R code

options(repr.plot.res = 300) # Adjust DPI as needed

# Create PheWAS Manhattan Plot
phewas_plot <- phewasManhattan(results, 
                               OR.direction = TRUE, 
                               title = sprintf("PheWAS Manhattan Plot %s", rs_name), 
                               annotate.size = 6, 
                               point.size = 6)

# Adjust x-axis label size and rotation
phewas_plot <- phewas_plot + theme(axis.text.x = element_text(size = 15, angle = 90))
# phewas_plot <- phewas_plot + geom_hline(yintercept = 4.4, linetype = "solid", color = "red", alpha = 0.5, size = 2)

# Remove gridlines
phewas_plot <- phewas_plot + theme(panel.grid.major = element_blank(), 
                                   panel.grid.minor = element_blank())

# Adjust y-axis limits
phewas_plot <- phewas_plot + coord_cartesian(ylim = c(0, 6))

# Step 3: Run dev.off() to create the file!
print(phewas_plot)
dev.off()